# Infer ***nuclei*** - 2️⃣ 

> WARNING: (🚨🚨🚨🚨 Steps 2-9 depend on establishing a good solution here.)

--------------

## OBJECTIVE: 
### ✅ Infer sub-cellular component #2: ***nuclei***  in order to understand interactome 

Infer a segmentation of the ***nuclei*** in order to measure its shape, position, size, and interaction with other organelles/cellular components.  

## OVERVIEW:

We will infer the nuclei using the nuclei channel (e.g. 'ch = 0).

Dependencies:
***Soma*** and ***cytoplasm*** inference rely on the ***nuclei*** inference.  Therefore all of the sub-cellular objects rely on the NU segmentation.


## preamble

1. imports
2. setup
3. infer-nuclei
    * input
    * pre-processing
    * core processing
    * post-processing
    * output



### IMPORTS

In [3]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict
from typing import Optional

import numpy as np

from aicssegmentation.core.pre_processing_utils import  image_smoothing_gaussian_slice_by_slice 
from aicssegmentation.core.utils import hole_filling
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))


from infer_subc.core.file_io import (read_czi_image,
                                                                    list_image_files)
from infer_subc.core.img import *
from infer_subc.organelles import fixed_get_optimal_Z_image, fixed_find_optimal_Z, find_optimal_Z
from infer_subc.constants import (TEST_IMG_N,
                                                                    NUC_CH ,
                                                                    LYSO_CH ,
                                                                    MITO_CH ,
                                                                    GOLGI_CH ,
                                                                    PEROX_CH ,
                                                                    ER_CH ,
                                                                    LD_CH ,
                                                                    RESIDUAL_CH, 
                                                                    ALL_CHANNELS )          

from infer_subc.organelles import infer_cellmask_fromaggr, fixed_infer_cellmask_fromaggr

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



## SETUP
CUSTOMIZE WITH: 
1. updated path to data
2. updated folder name for "raw" data

> NOTE: we are operating on a single "test" image in this notebook.  The batch-processing of all the images will be happen at the end of the notebook after we have developed/confirmed the setmentation procedures and parameter settings.

In [5]:
# this will be the example image for testing the pipeline below
test_img_n = TEST_IMG_N

# build the datapath
# all the imaging data goes here.
# CUSTOMIZE HERE --->
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python Scripts/infer-subc"
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
# CUSTOMIZE HERE --->
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files in "raw"
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[test_img_n]


In [6]:
# isolate image as an ndarray and metadata as a dictionary
img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


## infer ***nuclei***

### summary of steps

➡️ INPUT
- channel 0

PRE-PROCESSING
- scale to min 0, max 1.0
- median Filter window 4
- gaussian 1.34

CORE-PROCESSING
  - threshold method minimum cross-entropy.  
    - objects 50-400 pixels, 
    - threshold smoothing scale: 1.34 (later 1 pixel
    - threshold correction factor: 0.9 (later 1.2 )
    - lower / upper bounds  (.1, 1)
    - log transformed thresholding

POST-PROCESSING
  - fill holes
  - remove small objects

OUTPUT ➡️ 
- mask of NUCLEI


> #### Note:  in later steps we will isolate individual cells for analysis. Here, all nuclei are segmented and retained.



#### NOTE:  using Allen Cell Segmenter  [Nucleophosmin](https://www.allencell.org/cell-observations/category/nucleophosmin) might be a good generic mechanism.  e.g.
-  [playground_npm1.ipynb](https://github.com/AllenInstitute/aics-segmentation/blob/master/lookup_table_demo/playground_npm1.ipynb) and [npm1.py](https://github.com/AllenInstitute/aics-segmentation/blob/master/aicssegmentation/structure_wrapper/seg_npm1.py) and [npm1_SR.py](https://github.com/AllenInstitute/aics-segmentation/blob/master/aicssegmentation/structure_wrapper/seg_npm1_SR.py)


## INPUT (prototype)

Get the "raw" signals we need to analyze as well as any other dependencies in "inferred" objects.  

In [7]:
###################
# INPUT
###################
# raw_nuclei = img_data[NUC_CH].copy()
raw_nuclei = select_channel_from_raw(img_data, NUC_CH)

print(raw_nuclei.shape)

(16, 768, 768)


## PRE-PROCESSING (prototype)

In [8]:
###################
# PRE_PROCESSING
###################           
nuclei_norm = min_max_intensity_normalization(raw_nuclei)

med_filter_size = 4   
nuclei_med = median_filter_slice_by_slice(nuclei_norm,
                                      size=med_filter_size)
print(nuclei_med.shape)

gaussian_smoothing_sigma = 1.34
nuclei_gaus = image_smoothing_gaussian_slice_by_slice(nuclei_med,
                                                      sigma=gaussian_smoothing_sigma)
print(nuclei_gaus.shape)


(16, 768, 768)
(16, 768, 768)


### VISUALIZE: the nuclei image after pre-processing
Use this to adjust median filter size and gaussian sigma above.

In [91]:
viewer = napari.Viewer()

In [92]:
viewer.add_image(
    raw_nuclei,
    scale=scale
)
viewer.add_image(
    nuclei_med,
    scale=scale
)
viewer.add_image(
    nuclei_gaus,
    scale=scale
)

<Image layer 'nuclei_gaus' at 0x1cf5b5642b0>

## CORE PROCESSING (prototype)


> #### ASIDE: Thresholding
> [Thresholding](https://en.wikipedia.org/wiki/Thresholding_%28image_processing%29) is used to create binary images. A threshold value determines the intensity value separating foreground pixels from background pixels. Foregound pixels are pixels brighter than the threshold value, background pixels are darker. In many cases, images can be adequately segmented by thresholding followed by labelling of *connected components*, which is a fancy way of saying "groups of pixels that touch each other".
> 
> Different thresholding algorithms produce different results. [Otsu's method](https://en.wikipedia.org/wiki/Otsu%27s_method) and [Li's minimum cross entropy threshold](https://scikit-image.org/docs/dev/auto_examples/developers/plot_threshold_li.html) are two common algorithms. Below, we use Li. You can use `skimage.filters.threshold_<TAB>` to find different thresholding methods.


In [87]:

###################
# CORE_PROCESSING
###################

threshold_factor = 0.9 #from cellProfiler
thresh_min = .1
thresh_max = 1.
li_thresholded = apply_log_li_threshold(nuclei_gaus, thresh_factor=threshold_factor, thresh_min=thresh_min, thresh_max=thresh_max)

print(li_thresholded.shape)


## **************NEED TO ADJUST THESE SETTINGS TO ISOLATE MORE OF THE TOP AND BOTTOM OF THE NUCLEI***************

(17, 704, 704)


### VISUALIZE: the nuclei image after core processing
Use this to adjust threshold parameters above.

In [88]:
viewer.add_image(
    li_thresholded,
    scale=scale
)

<Image layer 'li_thresholded' at 0x1cf8f31fe80>

## POST PROCESSING prototype

> NOTE: the size parameters are by convention defined as one dimensional "width", so the inputs to the functions need to be _squared_ i.e. raised to the power of 2: `** 2`.   For volumetric (3D) analysis this would be _cubed_:`**3`

In [89]:
###################
# POST_PROCESSING
###################

hole_width = 5  
removed_holes = hole_filling(li_thresholded,
                             hole_min=0, 
                             hole_max=hole_width**2, 
                             fill_2d=True)
# there does not seem to be any observable differences between the slice-by-slice and 3D methods here for hole filling

small_object_width = 15
cleaned_img = size_filter(removed_holes, 
                          min_size= small_object_width**3, #changed this to 3 to adjust for the 3D voxel, instead of a 2D pixel 
                          method="3D",
                          connectivity=1)

## ************NEED TO ADJUST THESE SETTINGS TO GET RID OF SMALL BITS*****************


### VISUALIZE: the nuclei image after post-processing
Use this to adjust the size parameters above.

In [90]:
viewer.add_image(
    removed_holes,
    scale=scale
)
viewer.add_image(
    cleaned_img,
    scale=scale
)

<Image layer 'cleaned_img' at 0x1cf8f2081c0>

In [45]:
viewer.close()

RuntimeError: wrapped C/C++ object of type QtViewer has been deleted

### Adjust naming and type for labels

In [46]:
NU_signal = raw_nuclei

# renaming semantic segmentation of nuclei
nuclei_object = cleaned_img

# creating instance segmentations for all nuclei in nuclei_object
NU_labels = label(cleaned_img)

### VISUALIZE final segmentations

In [51]:
viewer = napari.Viewer()

In [52]:
viewer.add_image(
    NU_signal,
    scale=scale,
)
viewer.add_image(
    nuclei_object,
    scale=scale,
    opacity=0.3,
)    
viewer.add_labels(
    NU_labels,
    scale=scale,
    opacity=0.3,
)

<Labels layer 'NU_labels' at 0x1cf5b1d7cd0>

In [53]:
viewer.close()

## DEFINE `_infer_nuclei` function

Based on the _prototyping_ above define the function to infer nuclei.  


In [58]:
##########################
#  _infer_nuclei
##########################
def _infer_nuclei_3D( in_img: np.ndarray,
                       median_sz: int, 
                       gauss_sig: float,
                       thresh_factor: float,
                       thresh_min: float,
                       thresh_max: float,
                       max_hole_w: int,
                       small_obj_w: int,
                       sz_filter_method: str
                     ) -> np.ndarray:
    """
    Procedure to infer 3D nuclei segmentation from multichannel z-stack input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels
    soma_mask: Optional[np.ndarray] = None
        mask
    median_sz: int
        width of median filter for signal
    gauss_sig: float
        sigma for gaussian smoothing of  signal
    thresh_factor: float
        adjustment factor for log Li threholding
    thresh_min: float
        abs min threhold for log Li threholding
    thresh_max: float
        abs max threhold for log Li threholding
    max_hole_w: int
        hole filling cutoff for nuclei post-processing
    small_obj_w: int
        minimum object size cutoff for nuclei post-processing
    sz_filter_method: str
        method for size filtering; either "3D" or "slice_by_slice"

    Returns
    -------------
    nuclei_object
        mask defined extent of NU
    
    """

    nuc_ch = NUC_CH
    nuclei = select_channel_from_raw(in_img, nuc_ch)


    ###################
    # PRE_PROCESSING
    ###################                
    nuclei = min_max_intensity_normalization(nuclei)
    nuclei = median_filter_slice_by_slice(nuclei,
                                          size=median_sz)
    nuclei = image_smoothing_gaussian_slice_by_slice(nuclei,
                                                     sigma=gauss_sig )


    ###################
    # CORE_PROCESSING
    ###################
    nuclei_object = apply_log_li_threshold(nuclei, 
                                           thresh_factor=thresh_factor, 
                                           thresh_min=thresh_min, 
                                           thresh_max=thresh_max)


    ###################
    # POST_PROCESSING
    ###################
    nuclei_object = hole_filling(nuclei_object, 
                                 hole_min=0, 
                                 hole_max=max_hole_w**2, 
                                 fill_2d=True)

    nuclei_object = size_filter(nuclei_object, 
                                min_size = small_obj_w**3, 
                                method = sz_filter_method,
                                connectivity=1)


    return nuclei_object


## DEFINE `_fixed_infer_nuclei` function


In [63]:
##########################
#  fixed_infer_nuclei
##########################
def _fixed_infer_nuclei_3D(in_img: np.ndarray) -> np.ndarray:
    """
    Procedure to infer cellmask from linearly unmixed input, with a *fixed* set of parameters for each step in the procedure.  i.e. "hard coded"

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels
    soma_mask: np.ndarray
        mask
 
    Returns
    -------------
    nuclei_object
        mask defined extent of NU
    
    """

    nuc_ch = NUC_CH
    median_sz = 4   
    gauss_sig = 1.34
    threshold_factor = 0.9
    thresh_min = 0.1
    thresh_max = 1.0
    max_hole_w = 5
    small_obj_w = 15
    sz_filter_method = "3D"

    return _infer_nuclei_3D( in_img,
                             median_sz,
                             gauss_sig,
                             threshold_factor,
                             thresh_min,
                             thresh_max,
                             max_hole_w,
                             small_obj_w,
                             sz_filter_method )


---------------------
## TEST `_infer_nuclei`  function defined above


In [64]:
_NU_object =  _fixed_infer_nuclei_3D(img_data) 

In [68]:
viewer = napari.Viewer()


In [69]:
viewer.add_image(
    NU_signal,
    scale=scale,
)
viewer.add_image(
    nuclei_object,
    scale=scale,
    opacity=0.3,
)    
viewer.add_labels(
    NU_labels,
    scale=scale,
    opacity=0.3,
)
viewer.add_image(
    _NU_object,
    scale=scale
)

<Image layer '_NU_object' at 0x1cf9214fa60>

In [70]:
viewer.close()

<code style="background:yellow;color:black">***WIP*** 2D-->3D transition stops here</code>

---------------------
# TEST `infer_nuclei_fromlabel` exported functions

> the prototype `_infer_nuclei` was copied to the [`.organelles.nuclei`](../infer_subc/organelles/nuclei.py) sub-module 
##
`infer_nuclei_fromlabel` procedure

Use the `infer_nuclei_fromlabel` function to infer the Nucleus and export it as an _ome.tif_ for easy reference.

In [ ]:
from infer_subc.organelles.nuclei import infer_nuclei_fromlabel, fixed_infer_nuclei

nuclei_object =  fixed_infer_nuclei(img_2D, soma_mask) 


## Visualize  2


In [ ]:
viewer.add_image(
    _NU_object,
    scale=scale,
    opacity=0.3,
)    


viewer.add_labels(
    label(_NU_object),
    scale=scale,
    opacity=0.3,
)


In [ ]:
viewer.add_image(
    nuclei_object,
    scale=scale,
    opacity=0.3,
)    


In [ ]:
from napari.utils.notebook_display import nbscreenshot

# viewer.dims.ndisplay = 3
# viewer.camera.angles = (-30, 25, 120)
nbscreenshot(viewer, canvas_only=True)


In [ ]:
viewer.close()

-----------------
Write the `infer_nuclei_fromlabel` spec to the widget json 

In [1]:
from infer_subc.organelles_config.helper import add_function_spec_to_widget_json

_fixed_infer_nuclei =  {
        "name": "infer nuclei  (fixed parameters)",
        "python::module": "infer_subc.organelles",
        "python::function": "fixed_infer_nuclei",
        "parameters": None
        }

_fixed_infer_nuclei

{'name': 'infer nuclei  (fixed parameters)',
 'python::module': 'infer_subc.organelles',
 'python::function': 'fixed_infer_nuclei',
 'parameters': None}

In [ ]:

add_function_spec_to_widget_json("fixed_infer_nuclei",_fixed_infer_nuclei)

In [ ]:

_infer_nuclei =  {
        "name": "infer nuclei",
        "python::module": "infer_subc.organelles",
        "python::function": "infer_nuclei_fromlabel",
        "parameters": {
                "median_sz": {
                        "widget_type": "slider",
                        "data_type": "int",
                        "min": 3,
                        "max": 15,
                        "increment": 1
                },
                "gauss_sig": {
                        "data_type": "float",
                        "increment": 0.25,
                        "max": 15.0,
                        "min": 1.25,
                        "widget_type": "slider"
                },
                "thresh_factor": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": 1.2,
                        "min": 0.6,
                        "widget_type": "slider"
                },
                "thresh_min": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": .9,
                        "min": 0.0,
                        "widget_type": "slider"
                },
                "thresh_max": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": 1.0,
                        "min": 0.1,
                        "widget_type": "slider"
                },
                "max_hole_w": {
                        "data_type": "int",
                        "increment": 1,
                        "max": 40,
                        "min": 4,
                        "widget_type": "slider"
                },           
                "small_obj_w": {
                        "data_type": "int",
                        "increment": 1,
                        "max": 50,
                        "min": 1,
                        "widget_type": "slider"
                }
        }
}

add_function_spec_to_widget_json("infer_nuclei_fromlabel", _infer_nuclei, overwrite=True )



In [ ]:
_median_filter_slice_by_slice =  {
                "name": "Median Smoothing Slice by Slice",
                "python::module": "infer_subc.core.img",
                "python::function": "median_filter_slice_by_slice",
                "parameters": {
                    "size": {
                        "widget_type": "slider",
                        "data_type": "int",
                        "min": 1,
                        "max": 20,
                        "increment": 1
                    }
                }
            } 
add_function_spec_to_widget_json("median_filter_slice_by_slice",_median_filter_slice_by_slice)

In [ ]:


_image_smoothing_gaussian_slice_by_slice = {
        "name": "Gaussian Smoothing Slice by Slice",
        "python::module": "aicssegmentation.core.pre_processing_utils",
        "python::function": "image_smoothing_gaussian_slice_by_slice",
        "parameters": {
            "sigma": {
                "widget_type": "slider",
                "data_type": "float",
                "min": 0.8,
                "max": 20,
                "increment": 0.2
            }
        }
        }

# json.dumps({"image_smoothing_gaussian_slice_by_slice": _image_smoothing_gaussian_slice_by_slice} )
add_function_spec_to_widget_json("image_smoothing_gaussian_slice_by_slice",_image_smoothing_gaussian_slice_by_slice)        


In [ ]:

# WARNING: not a good way to set to None
_apply_log_li_threshold = {
        "name": "threshold log Li",
        "python::module": "infer_subc.core.img",
        "python::function": "apply_log_li_threshold",
        "parameters": {
            "thresh_factor": {
                "widget_type": "slider",
                "data_type": "float",
                "min": 0.3,
                "max": 1.1,
                "increment": 0.05
            },
            "thresh_min": {
                "widget_type": "slider",
                "data_type": "float",
                "min": 0.0,
                "max": 0.8,
                "increment": 0.01
            },
            "thresh_max": {
                "widget_type": "slider",
                "data_type": "float",
                "min": 0.3,
                "max": 1.0,
                "increment": 0.05
            },
        }
        }

# json.dumps({"apply_log_li_threshold": _apply_log_li_threshold} )
add_function_spec_to_widget_json("apply_log_li_threshold",_apply_log_li_threshold,overwrite=True)        


In [ ]:


    # NU_labels = label(nuclei_object)

_label =  {
        "name": "label objects",
        "python::module": "skimage.measure",
        "python::function": "label",
        "parameters": None
        }
# json.dumps({"label":_label})
add_function_spec_to_widget_json("label",_label)        


In [ ]:

#  nulei_object = apply_mask(nuclei_object, soma_mask)

_apply_mask=  {
        "name": "apply mask",
        "python::module": "infer_subc.core.img",
        "python::function": "apply_mask",
        "parameters": None
        }
# json.dumps({"apply_mask":_apply_mask})
add_function_spec_to_widget_json("apply_mask",_apply_mask)        


In [ ]:

    # small_object_width = 45
    # nuclei_object = size_filter_linear_size(nuclei_object, 
    #                                                             min_size= small_object_width**2, 
    #                                                             connectivity=1)


_size_filter_linear_size = {
        "name": "Size Filter 2D",
        "python::module": "infer_subc.core.img",
        "python::function": "size_filter_linear_size",
        "parameters": {
            "min_size": {
                "widget_type": "slider",
                "data_type": "int",
                "min": 0,
                "max": 500,
                "increment": 1
            }
        }
    }
# json.dumps({  "size_filter_linear_size":  _size_filter_linear_size   })

add_function_spec_to_widget_json("size_filter_linear_size",_size_filter_linear_size)        


-------------------------------
## Write workflow .json
Now that we've added our function specs we can compose workflows.

In [ ]:
from infer_subc.constants import NUC_CH


def make_infer_nuclei_dict():
    """
    Procedure to infer nuclei from linearly unmixed input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels

    soma_mask: np.ndarray
        mask

    Returns
    -------------
    nuclei_object
        mask defined extent of NU

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   
    step_name.append("1")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = NUC_CH) )
    parent.append(0)

    ###################
    # PRE_PROCESSING
    ###################                         
    # nuclei = min_max_intensity_normalization(in_img[NUC_CH].copy() )
    step_name.append("2")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(1)

    # size = 4   
    # nuclei = median_filter_slice_by_slice( 
    #                                                                 nuclei,
    #                                                                 size=size  )
    step_name.append("3")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 4 ))
    parent.append(2)

    # sigma = 1.34
    # truncate_range = 3.0
    # nuclei = image_smoothing_gaussian_slice_by_slice(  nuclei,
    #                                                                                             sigma=sigma,
    #                                                                                             truncate_range = truncate_range
    #                                                                                             )
    step_name.append("4")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.34 ))
    parent.append(3)

    ###################
    # CORE_PROCESSING
    ###################
    # threshold_factor = 0.9 
    # thresh_min = .1
    # thresh_max = 1.
    # nuclei_object = apply_log_li_threshold(nuclei, threshold_factor=threshold_factor, thresh_min=thresh_min, thresh_max=thresh_max)
    step_name.append("5")
    function_name.append("apply_log_li_threshold")
    category.append("core")
    parameter_values.append(dict(thresh_factor = 0.9, 
                                                            thresh_min = .1,
                                                            thresh_max = 1.))
    parent.append(4)


    # NU_labels = label(nuclei_object)
    step_name.append("6")
    function_name.append("label")
    category.append("core")
    parameter_values.append(None)
    parent.append(5)


    ###################
    # POST_PROCESSING
    ###################
    # hole_width = 5  
    # nuclei_object = hole_filling(nuclei_object, hole_min=0, hole_max=hole_width**2, fill_2d=True)
    step_name.append("6")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=5**2, fill_2d=True))
    parent.append(5)

    # # EEEEEK I don't know how to compose where the mask comes from... 
    # nuclei_object = apply_mask(nuclei_object, soma_mask)

    # small_object_width = 15
    # nuclei_object = size_filter_linear_size(nuclei_object, 
    #                                                             min_size= small_object_width**2, 
    #                                                             connectivity=1)
    step_name.append("7")
    function_name.append("size_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 15**2  ))
    parent.append(6)

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:
from infer_subc.utils.directories import Directories
import json

def _write_workflow_json(wf_name, wf_dict):

    # read all_functions.json into dict
    # if not wf_name.startswith("conf"):
    #     wf_name = f"conf_{wf_name}"
    path = Directories.get_structure_config_dir() / f"{wf_name}.json"

    # re-write file
    with open(path, "w") as file:
        json.dump(wf_dict, file, indent=4, sort_keys=False)

    return path




In [ ]:
from infer_subc.organelles_config.helper import write_workflow_json

infer_nuclei_dict = make_infer_nuclei_dict()

write_workflow_json("conf_2.2.nuclei_stepbystep", infer_nuclei_dict)

In [ ]:
from infer_subc.constants import NUC_CH


def make_infer_nuclei_from_raw_dict():
    """
    Procedure to infer nuclei from linearly unmixed input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels

    soma_mask: np.ndarray
        mask

    Returns
    -------------
    nuclei_object
        mask defined extent of NU

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   

    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)


    step_name.append("2")
    function_name.append("fixed_infer_cellmask_fromaggr")
    category.append("extraction")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = NUC_CH) )
    parent.append(1)

    ###################
    # PRE_PROCESSING
    ###################                         
    # nuclei = min_max_intensity_normalization(in_img[NUC_CH].copy() )
    step_name.append("4")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(3)

    # size = 4   
    # nuclei = median_filter_slice_by_slice( 
    #                                                                 nuclei,
    #                                                                 size=size  )
    step_name.append("5")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 4 ))
    parent.append(4)

    # sigma = 1.34
    # truncate_range = 3.0
    # nuclei = image_smoothing_gaussian_slice_by_slice(  nuclei,
    #                                                                                             sigma=sigma,
    #                                                                                             truncate_range = truncate_range
    #                                                                                             )
    step_name.append("6")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.34 ))
    parent.append(5)

    ###################
    # CORE_PROCESSING
    ###################
    # threshold_factor = 0.9 
    # thresh_min = .1
    # thresh_max = 1.
    # nuclei_object = apply_log_li_threshold(nuclei, threshold_factor=threshold_factor, thresh_min=thresh_min, thresh_max=thresh_max)
    step_name.append("7")
    function_name.append("apply_log_li_threshold")
    category.append("core")
    parameter_values.append(dict(thresh_factor = 0.9, 
                                                            thresh_min = .1,
                                                            thresh_max = 1.))
    parent.append(6)


    # NU_labels = label(nuclei_object)
    step_name.append("8")
    function_name.append("label")
    category.append("core")
    parameter_values.append(None)
    parent.append(7)


    ###################
    # POST_PROCESSING
    ###################
    # hole_width = 5  
    # nuclei_object = hole_filling(nuclei_object, hole_min=0, hole_max=hole_width**2, fill_2d=True)
    step_name.append("9")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=5**2, fill_2d=True))
    parent.append(8)

    step_name.append("10")
    function_name.append("apply_mask")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([9,2])


    # small_object_width = 15
    # nuclei_object = size_filter_linear_size(nuclei_object, 
    #                                                             min_size= small_object_width**2, 
    #                                                             connectivity=1)
    step_name.append("11")
    function_name.append("size_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 15**2  ))
    parent.append(10)

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:
from infer_subc.organelles_config.helper import write_workflow_json

infer_nuclei_dict = make_infer_nuclei_from_raw_dict()

write_workflow_json("conf_1.2.nuclei_stepbystep_from_raw", infer_nuclei_dict)

In [ ]:
from infer_subc.organelles_config.helper import add_function_spec_to_widget_json

_infer_nuclei =  {
        "name": "infer infer_nuclei_fromlabel",
        "python::module": "infer_subc.organelles",
        "python::function": "infer_nuclei_fromlabel",
        "parameters": None
        }

add_function_spec_to_widget_json("infer_nuclei_fromlabel",_infer_nuclei)

-------------
## SUMMARY

The above details how the nuclei object is inferred.  

### NEXT: INFER CYTOSOL

proceed to [03_infer_cytoplasm.ipynb](./03_infer_cytoplasm.ipynb)
